# Подготовка данных

In [1]:
%matplotlib inline
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
import sklearn
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
# df = pd.read_csv(r"InternationalBifurca_DATA_2023-10-30_0629.csv", sep=',')
df = pd.read_csv(r"InternationalBifurca_DATA_2025-04-20_0932.csv", sep=',')

/var/folders/2x/tx689lxn54ngmttr2b89rvg00000gn/T/ipykernel_30336/3738330455.py:2: DtypeWarning: Columns (264) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"InternationalBifurca_DATA_2025-04-20_0932.csv", sep=',')


In [3]:
df = df.dropna(subset=['sex'])
df

,record_id,identifier,date,adhoc_pci,sex,age,weight,height,clinical_presentation,time_from_mi_symptoms_onse,...,time_to_death_f5,time_to_acs_f5,time_to_stroke_f5,time_to_pci_f5,time_to_cabg_f5,hospitalization_f5,bleeding_f5,angio_follow_f5,restenosis_f5,side_branch_restenosis_5
0,MNRI0001,MNRI-2018-0001,2018-02-01,1.0,2.0,77.0,84.0,165.0,5.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MNRI0002,MNRI-2018-0002,2018-01-24,0.0,1.0,68.0,81.0,171.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MNRI0003,MNRI-2018-0003,2018-01-24,0.0,1.0,62.0,74.0,180.0,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MNRI0004,MNRI-2018-0004,2018-01-30,1.0,1.0,67.0,84.0,167.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MNRI0005,MNRI-2018-0005,2018-01-30,0.0,1.0,57.0,103.0,174.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2139,TRCH0026,TRCH-2019-0026,2019-03-11,1.0,1.0,67.0,90.0,174.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2140,TRCH0027,TRCH-2019-0027,2019-03-18,1.0,1.0,69.0,60.0,174.0,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2141,TRCH0028,TRCH-2019-0028,2019-03-19,0.0,2.0,81.0,50.0,160.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2142,TRCH0029,TRCH-2019-0029,2019-03-28,1.0,1.0,86.0,74.0,170.0,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
anyInf = df[df == np.inf].sum()
anyInf[anyInf != 0]

stent_distal_vessel_size    inf
sb_stent_sb_diametr         inf
dtype: object

In [5]:
patient_info_cols = ['record_id', 'date', 'sex', 'age', 'adhoc_pci', 'weight', 'height', 
                     'clinical_presentation', 'time_from_mi_symptoms_onse', 
                     'ccs_class', 'diabet', 'insulin_diabetes', 'hypertension', 'smoking', 
                     'dyslipidemia', 'anemia', 'atrial_fibrilation', 'oac_use', 'valvular_disease', 
                     'valvular_disease_was_previ', 'if_yes_what_type___1', 'if_yes_what_type___2', 
                     'if_yes_what_type___3', 'if_yes_what_type___4', 'if_yes_what_type___5', 
                     'if_yes_what_type___6', 'if_yes_what_type___7', 'ef', 'creatinine', 'ckd', 
                     'mi_history', 'cerebrovascular_disease', 'previously_treated_cerebro', 'previous_stroke_tia', 
                     'peripheral_artery_disease', 'previously_treated_periphe', 'copd', 'history_of_cancer', 
                     'previous_pci', 'previous_cabg']

intervention_cols = ['single_vessel', 'trifurcation', 'bifurcation_location', 
                  'lesion_ivolves', 'angle', 'calcium', 'trombosis', 
                  'total_trobotic_occlusion', 'restenosis_reocclusion', 'cto_bifurc', 
                  'medina_proximal', 'medina_distal', 'medina_side', 'mb_length_proximal', 
                  'sb_length', 'proximal_diametr', 'distal_diametr', 'side_diametr', 'stenosis_proximal', 
                  'stenosis_distal', 'timi_flow_main_branch', 'side_stenosis', 'timi_flow_side_branch', 
                  'major_lm', 'major_non_lm', 'minor_criteria', 'main_branch_rvd', 'def', 'def_2']

operation_cols = ['side_protection', 'main_predilatation', 'side_predilat', 
                  'stent_was_implated_from_lm', 'stent_number', 'stent_number_bif', 'stent_technique', 
                  'first_stent_impanted', 'provisional_2_stent_techni', 'stent_direction', 'defered_stenting', 
                  'stent_diameter', 'stent_length', 'stent_type___1', 'stent_type___2', 'stent_type___3', 
                  'stent_type___4', 'stent_type___5', 'stent_type___6', 'stent_type___7', 'stent_type___9', 
                  'stent_type___8', 'dstent2', 'stent_length2', 'stent_distal_vessel_size', 
                  'sb_stent_side_branch_diametr', 'sb_stent_sb_diametr', 'twostent_technique', 
                  'sb_dilatation', 'stent_postdilatation', 'proximal_optimization', 'pot', 
                  'pot_balloon_diametr', 'kissing_post', 'modified_kis', 'several_kissing']

new_cols = ['adverse_event_followup_f2_v2', 'angio_follow_f5', 'antiplatalet_drug_was_chan',
            'attempt_to_dilate_stenting', 'ballooon_size_for_postdila', 'complete_revascularisation',
            'currently_on_dialysis', 'followup_1_year_do_not_complete_if_2nd_bifurcation_complete',
            'identifier', 'ishemia_test___1', 'ishemia_test___2', 'ishemia_test___3', 'ishemia_test___4',
            'kissing_post_2stent___1', 'kissing_post_2stent___2', 'left_main_stent_direction',
            'main_branch_calcification', 'mb_stenosis_f2', 'medina_side_branch_2', 'myocardial_ischemia',
            'myocardial_ishemia_was_det', 'number_of_kissing', 'number_of_kissing_2',
            'other_lesions_in_main_bran', 'other_lesions_in_side_brach',
            'patient_information_do_not_complete_if_2nd_bifurca_complete', 'pot_2', 'pot_balloon_diametr_2',
            'pot_balloon_length', 'pot_balloon_length_2', 'pressure2', 'reson_for_change_stopped___1',
            'reson_for_change_stopped___2', 'reson_for_change_stopped___3', 'restenosis_f5', 'sb_length_2',
            'sb_stenosis_f2', 'side_branch_calcification_2', 'side_branch_restenosis',
            'side_branch_restenosis_3', 'side_branch_restenosis_5', 'stent_pressure', 'stent_type_2___1',
            'stent_type_2___2', 'stent_type_2___3', 'stent_type_2___4', 'stent_type_2___5', 'stent_type_2___6',
            'stent_type_2___7', 'stent_type_2___8', 'stent_type_2___9', 'thrombolysis', 'uncross_strategy___1',
            'uncross_strategy___2', 'uncross_strategy___3', 'uncross_strategy___4', 'uncross_strategy___5',
            'uncross_strategy___6', 'uncross_strategy___7', 'uncross_strategy___8', 'uncross_strategy___9']

In [6]:
non_vascular_deaths = ['MNRI1054', 'MNRI1191', 'MNRI1351', 'MNRI1352', 'MNRI1473', 'MNRI1670', 'MNRI0637', 'MNRI0656', 'MNRI0751', 'MNRI0758',
                      'MNRI0805', 'MNRI0818', 'MNRI1054', 'MNRI0087', 'MNRI1191', 'MNRI0108', 'MNRI0307', 'MNRI0215', 'MNRI0322', 'MNRI0293',
                      'MNRI0156', 'MNRI0215', 'MNRI0488', 'MNRI0612', 'MNRI0708', 'MNRI0767', 'MNRI0772', 'MNRI0786', 'MNRI1105', 'MNRI1186',
                      'MNRI1462', 'MNRI1633']

In [7]:
df = df[~df['record_id'].isin(non_vascular_deaths)]

In [8]:
df

,record_id,identifier,date,adhoc_pci,sex,age,weight,height,clinical_presentation,time_from_mi_symptoms_onse,...,time_to_death_f5,time_to_acs_f5,time_to_stroke_f5,time_to_pci_f5,time_to_cabg_f5,hospitalization_f5,bleeding_f5,angio_follow_f5,restenosis_f5,side_branch_restenosis_5
0,MNRI0001,MNRI-2018-0001,2018-02-01,1.0,2.0,77.0,84.0,165.0,5.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MNRI0002,MNRI-2018-0002,2018-01-24,0.0,1.0,68.0,81.0,171.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MNRI0003,MNRI-2018-0003,2018-01-24,0.0,1.0,62.0,74.0,180.0,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MNRI0004,MNRI-2018-0004,2018-01-30,1.0,1.0,67.0,84.0,167.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MNRI0005,MNRI-2018-0005,2018-01-30,0.0,1.0,57.0,103.0,174.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2139,TRCH0026,TRCH-2019-0026,2019-03-11,1.0,1.0,67.0,90.0,174.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2140,TRCH0027,TRCH-2019-0027,2019-03-18,1.0,1.0,69.0,60.0,174.0,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2141,TRCH0028,TRCH-2019-0028,2019-03-19,0.0,2.0,81.0,50.0,160.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2142,TRCH0029,TRCH-2019-0029,2019-03-28,1.0,1.0,86.0,74.0,170.0,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
patient_present_cols = patient_info_cols + intervention_cols + operation_cols + new_cols
patient_present_df = pd.DataFrame({col_name: df[col_name] for col_name in patient_present_cols})

In [10]:
patient_present_df

,record_id,date,sex,age,adhoc_pci,weight,height,clinical_presentation,time_from_mi_symptoms_onse,ccs_class,...,thrombolysis,uncross_strategy___1,uncross_strategy___2,uncross_strategy___3,uncross_strategy___4,uncross_strategy___5,uncross_strategy___6,uncross_strategy___7,uncross_strategy___8,uncross_strategy___9
0,MNRI0001,2018-02-01,2.0,77.0,1.0,84.0,165.0,5.0,4.0,NaN,...,0.0,0,0,0,0,0,0,0,0,0
1,MNRI0002,2018-01-24,1.0,68.0,0.0,81.0,171.0,1.0,NaN,1.0,...,NaN,0,0,0,0,0,0,0,0,0
2,MNRI0003,2018-01-24,1.0,62.0,0.0,74.0,180.0,4.0,NaN,NaN,...,NaN,0,0,0,0,0,0,0,0,0
3,MNRI0004,2018-01-30,1.0,67.0,1.0,84.0,167.0,2.0,NaN,NaN,...,NaN,0,0,0,0,0,0,0,0,0
4,MNRI0005,2018-01-30,1.0,57.0,0.0,103.0,174.0,1.0,NaN,2.0,...,NaN,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2139,TRCH0026,2019-03-11,1.0,67.0,1.0,90.0,174.0,2.0,NaN,NaN,...,0.0,0,0,0,0,0,0,0,0,0
2140,TRCH0027,2019-03-18,1.0,69.0,1.0,60.0,174.0,3.0,1.0,NaN,...,1.0,0,0,0,0,0,0,0,0,0
2141,TRCH0028,2019-03-19,2.0,81.0,0.0,50.0,160.0,2.0,NaN,NaN,...,0.0,0,0,0,0,0,0,0,0,0
2142,TRCH0029,2019-03-28,1.0,86.0,1.0,74.0,170.0,3.0,1.0,NaN,...,0.0,0,0,0,0,0,0,0,0,0


In [11]:
def remove_columns_with_nan_threshold(df, threshold=250):
    columns_to_preserve = ['valvular_disease', 'previous_stroke_tia', 'twostent_technique']
    
    nan_counts = df.isnull().sum()
    
    columns_to_drop = [col for col in nan_counts[nan_counts > threshold].index 
                      if col not in columns_to_preserve]
    
    print(columns_to_drop)
    
    df_dropped = df.drop(columns=columns_to_drop)
    
    return df_dropped

In [12]:
patient_present_df = remove_columns_with_nan_threshold(patient_present_df, 500)

['time_from_mi_symptoms_onse', 'ccs_class', 'insulin_diabetes', 'valvular_disease_was_previ', 'previously_treated_cerebro', 'previously_treated_periphe', 'lesion_ivolves', 'total_trobotic_occlusion', 'sb_length', 'timi_flow_main_branch', 'timi_flow_side_branch', 'stent_was_implated_from_lm', 'first_stent_impanted', 'provisional_2_stent_techni', 'dstent2', 'stent_length2', 'sb_stent_side_branch_diametr', 'proximal_optimization', 'pot', 'pot_balloon_diametr', 'several_kissing', 'adverse_event_followup_f2_v2', 'angio_follow_f5', 'antiplatalet_drug_was_chan', 'attempt_to_dilate_stenting', 'ballooon_size_for_postdila', 'complete_revascularisation', 'left_main_stent_direction', 'main_branch_calcification', 'mb_stenosis_f2', 'medina_side_branch_2', 'myocardial_ischemia', 'myocardial_ishemia_was_det', 'number_of_kissing', 'number_of_kissing_2', 'other_lesions_in_main_bran', 'other_lesions_in_side_brach', 'pot_2', 'pot_balloon_diametr_2', 'pot_balloon_length', 'pot_balloon_length_2', 'pressure2

In [13]:
# patient_present_df = patient_present_df.drop(['record_id', 'date'], axis = 1)
patient_present_df = patient_present_df.drop(['date'], axis = 1)
patient_present_df

,record_id,sex,age,adhoc_pci,weight,height,clinical_presentation,diabet,hypertension,smoking,...,stent_type_2___9,uncross_strategy___1,uncross_strategy___2,uncross_strategy___3,uncross_strategy___4,uncross_strategy___5,uncross_strategy___6,uncross_strategy___7,uncross_strategy___8,uncross_strategy___9
0,MNRI0001,2.0,77.0,1.0,84.0,165.0,5.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,MNRI0002,1.0,68.0,0.0,81.0,171.0,1.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,MNRI0003,1.0,62.0,0.0,74.0,180.0,4.0,0.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
3,MNRI0004,1.0,67.0,1.0,84.0,167.0,2.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,MNRI0005,1.0,57.0,0.0,103.0,174.0,1.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2139,TRCH0026,1.0,67.0,1.0,90.0,174.0,2.0,2.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2140,TRCH0027,1.0,69.0,1.0,60.0,174.0,3.0,1.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2141,TRCH0028,2.0,81.0,0.0,50.0,160.0,2.0,2.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2142,TRCH0029,1.0,86.0,1.0,74.0,170.0,3.0,1.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
patient_present_df.replace(to_replace = [np.inf, -np.inf], value= None, inplace=True)

In [15]:
cols_to_drop = patient_present_df.columns[patient_present_df.nunique() <= 1]
patient_present_df = patient_present_df.drop(cols_to_drop, axis=1)
patient_present_df

,record_id,sex,age,adhoc_pci,weight,height,clinical_presentation,diabet,hypertension,smoking,...,stent_type_2___5,stent_type_2___6,stent_type_2___7,stent_type_2___8,uncross_strategy___1,uncross_strategy___2,uncross_strategy___3,uncross_strategy___4,uncross_strategy___8,uncross_strategy___9
0,MNRI0001,2.0,77.0,1.0,84.0,165.0,5.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,MNRI0002,1.0,68.0,0.0,81.0,171.0,1.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,MNRI0003,1.0,62.0,0.0,74.0,180.0,4.0,0.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
3,MNRI0004,1.0,67.0,1.0,84.0,167.0,2.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,MNRI0005,1.0,57.0,0.0,103.0,174.0,1.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2139,TRCH0026,1.0,67.0,1.0,90.0,174.0,2.0,2.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2140,TRCH0027,1.0,69.0,1.0,60.0,174.0,3.0,1.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2141,TRCH0028,2.0,81.0,0.0,50.0,160.0,2.0,2.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2142,TRCH0029,1.0,86.0,1.0,74.0,170.0,3.0,1.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
numerical = ['age', 'weight', 'height', 'ef', 'creatinine', 'ckd', 'angle', 'mb_length_proximal', 
            'proximal_diametr', 'distal_diametr', 'side_diametr', 'stenosis_proximal', 
            'stenosis_distal', 'side_stenosis', 'minor_criteria', 'main_branch_rvd', 
            'stent_diameter', 'stent_length', 'stent_distal_vessel_size', 'sb_stent_sb_diametr',
            # New numerical columns based on the analysis
            'ballooon_size_for_postdila', 
            'left_main_stent_direction',
            'mb_stenosis_f2',
            'myocardial_ischemia',
            'number_of_kissing_2',
            'pot_balloon_diametr_2',
            'pot_balloon_length',
            'pot_balloon_length_2',
            'pressure2',
            'sb_length_2',
            'sb_stenosis_f2',
            'stent_pressure']

categorical = ['sex', 'clinical_presentation', 'bifurcation_location', 'stent_number', 
              'stent_number_bif', 'stent_technique', 'stent_direction']

binary = ['diabet', 'adhoc_pci', 'hypertension', 'smoking', 'dyslipidemia', 'anemia', 
         'atrial_fibrilation', 'oac_use', 'if_yes_what_type___1', 'if_yes_what_type___2',
         'if_yes_what_type___3', 'if_yes_what_type___4', 'if_yes_what_type___6', 
         'mi_history', 'cerebrovascular_disease', 'peripheral_artery_disease', 'copd', 
         'history_of_cancer', 'previous_pci', 'previous_cabg', 'single_vessel', 'trifurcation',
         'calcium', 'trombosis', 'restenosis_reocclusion', 'cto_bifurc', 
         'medina_proximal', 'medina_distal', 'medina_side', 'major_lm', 'major_non_lm',
         'def', 'def_2', 'side_protection', 'main_predilatation', 'side_predilat',
         'defered_stenting', 'stent_type___1', 'stent_type___2', 'stent_type___3', 'stent_type___4',
         'stent_type___5', 'stent_type___6', 'stent_type___7', 'stent_type___9', 'stent_type___8', 
         'sb_dilatation', 'stent_postdilatation', 'kissing_post', 'modified_kis',
         # New binary columns based on the analysis
         'currently_on_dialysis',
         'ishemia_test___1',
         'ishemia_test___2',
         'ishemia_test___3',
         'kissing_post_2stent___1',
         'kissing_post_2stent___2',
         'reson_for_change_stopped___1',
         'reson_for_change_stopped___2',
         'reson_for_change_stopped___3',
         'stent_type_2___1',
         'stent_type_2___3',
         'stent_type_2___4',
         'stent_type_2___5',
         'stent_type_2___6',
         'stent_type_2___7',
         'stent_type_2___8',
         'uncross_strategy___1',
         'uncross_strategy___2',
         'uncross_strategy___3',
         'uncross_strategy___4',
         'uncross_strategy___8',
         'uncross_strategy___9']

In [17]:
without_second_bif = df
adverse_events = without_second_bif['event_type_followup_f2___1'] \
| without_second_bif['event_type_followup_f2___2'] \
| without_second_bif['event_type_followup_f2_v2___1'] \
| without_second_bif['event_type_followup_f2_v2___2']
print(sum(adverse_events))
print(len(adverse_events))

158
2044


In [18]:
combined = zip(without_second_bif['event_type_followup_f2___1'],  
               without_second_bif['event_type_followup_f2___2'] * 2,
               without_second_bif['event_type_followup_f2_v2___1'] * 3, 
               without_second_bif['event_type_followup_f2_v2___2'] * 4)

combined_adverse_events = np.array([max(t) for t in combined])

In [19]:
id_col = patient_present_df['record_id']
patient_present_df = patient_present_df.drop(['record_id', 'identifier'], axis=1)

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(patient_present_df, combined_adverse_events, test_size=0.4, stratify=combined_adverse_events, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, stratify=y_test, random_state=42)

In [21]:
y_train[y_train != 0] = 1
y_test[y_test != 0] = 1
y_val[y_val != 0] = 1

In [22]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer

imputer_categorical = SimpleImputer(strategy='most_frequent')
X_train[categorical + binary] = imputer_categorical.fit_transform(X_train[categorical + binary])
X_test[categorical + binary] = imputer_categorical.transform(X_test[categorical + binary])
X_val[categorical + binary] = imputer_categorical.transform(X_val[categorical + binary])

imputer = IterativeImputer(random_state=0)
columns = list(X_train.columns)
X_train = pd.DataFrame(data = imputer.fit_transform(X_train), columns = columns)
X_test = pd.DataFrame(data = imputer.transform(X_test), columns = columns)
X_val = pd.DataFrame(data = imputer.transform(X_val), columns = columns)

X_train

,sex,age,adhoc_pci,weight,height,clinical_presentation,diabet,hypertension,smoking,dyslipidemia,...,stent_type_2___5,stent_type_2___6,stent_type_2___7,stent_type_2___8,uncross_strategy___1,uncross_strategy___2,uncross_strategy___3,uncross_strategy___4,uncross_strategy___8,uncross_strategy___9
0,1.0,41.0,0.0,100.000000,177.000000,5.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,56.0,1.0,81.000000,172.000000,4.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,55.0,1.0,87.535524,172.093993,3.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,79.0,0.0,61.000000,160.000000,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1.0,61.0,1.0,84.000000,179.000000,2.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,2.0,69.0,0.0,111.000000,159.000000,1.0,2.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1222,1.0,57.0,0.0,96.000000,178.000000,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1223,1.0,63.0,0.0,80.000000,162.000000,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1224,1.0,75.0,0.0,165.000000,71.000000,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse_output=False)
df_for_ohe = pd.concat([X_train[categorical], X_test[categorical], X_val[categorical]], ignore_index=True)
ohe.fit(df_for_ohe)

OneHotEncoder(sparse_output=False)

In [24]:
def process_ohe(dataframe, cat_cols, encoder):
    encoded_columns = pd.DataFrame(encoder.transform(dataframe[cat_cols]))

    encoded_columns.columns = encoder.get_feature_names_out(cat_cols)

    dataframe = pd.concat([dataframe, encoded_columns], axis=1)

    dataframe.drop(cat_cols, axis=1, inplace=True)

    return dataframe

In [25]:
X_train = process_ohe(X_train, categorical, ohe)
X_test = process_ohe(X_test, categorical, ohe)
X_val = process_ohe(X_val, categorical, ohe)
X_train

,age,adhoc_pci,weight,height,diabet,hypertension,smoking,dyslipidemia,anemia,atrial_fibrilation,...,stent_number_bif_3.0,stent_number_bif_4.0,stent_technique_0.0,stent_technique_1.0,stent_direction_1.0,stent_direction_2.0,stent_direction_3.0,stent_direction_4.0,stent_direction_5.0,stent_direction_6.0
0,41.0,0.0,100.000000,177.000000,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,56.0,1.0,81.000000,172.000000,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,55.0,1.0,87.535524,172.093993,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,79.0,0.0,61.000000,160.000000,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,61.0,1.0,84.000000,179.000000,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,69.0,0.0,111.000000,159.000000,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1222,57.0,0.0,96.000000,178.000000,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1223,63.0,0.0,80.000000,162.000000,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1224,75.0,0.0,165.000000,71.000000,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [26]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

feature_selector = SelectKBest(f_classif, k=40)
X_feature_selection = feature_selector.fit_transform(X_train, y_train)
X_feature_selection.shape

strong_cols = []

feature_scores = feature_selector.scores_
features = X_train.columns
features_scores_sorted = sorted(zip(features, feature_scores), key=lambda x: x[1], reverse=True)
for col in features_scores_sorted[:30]:
    strong_cols.append(col[0])

/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:111: UserWarning: Features [77] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [27]:
X_train = X_train[strong_cols]
X_test = X_test[strong_cols]
X_val = X_val[strong_cols]
X_train

,age,anemia,ef,cerebrovascular_disease,ckd,peripheral_artery_disease,if_yes_what_type___1,creatinine,single_vessel,calcium,...,stent_diameter,minor_criteria,stent_length,adhoc_pci,previous_pci,stent_type___4,cto_bifurc,valvular_disease,side_diametr,major_lm
0,41.0,0.0,49.0,0.0,91.130343,0.0,0.0,90.0,0.0,0.0,...,4.00,1.0,11.0,0.0,1.0,0.0,0.0,0.000000,2.4,0.0
1,56.0,0.0,68.0,0.0,83.132035,0.0,0.0,89.0,1.0,0.0,...,2.75,0.0,18.0,1.0,1.0,1.0,0.0,0.000000,3.0,0.0
2,55.0,0.0,70.0,0.0,78.364633,0.0,0.0,94.0,1.0,0.0,...,2.75,2.0,33.0,1.0,0.0,1.0,0.0,0.000000,2.4,0.0
3,79.0,0.0,73.0,1.0,64.542788,0.0,0.0,96.0,0.0,1.0,...,3.50,3.0,38.0,0.0,0.0,0.0,0.0,0.000000,3.1,0.0
4,61.0,0.0,62.0,0.0,87.329657,0.0,0.0,83.0,0.0,0.0,...,4.00,2.0,28.0,1.0,0.0,0.0,0.0,0.000000,2.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,69.0,0.0,60.0,0.0,69.608441,0.0,0.0,97.0,1.0,0.0,...,3.50,0.0,32.0,0.0,1.0,0.0,0.0,-0.048430,2.7,0.0
1222,57.0,0.0,39.0,0.0,53.707072,0.0,0.0,127.0,0.0,0.0,...,3.50,2.0,23.0,0.0,0.0,0.0,0.0,0.005881,2.4,0.0
1223,63.0,1.0,71.0,0.0,86.111321,0.0,0.0,83.0,1.0,0.0,...,2.75,2.0,18.0,0.0,1.0,0.0,0.0,0.000000,2.2,0.0
1224,75.0,0.0,69.0,1.0,68.979615,1.0,0.0,93.0,1.0,0.0,...,3.00,0.0,18.0,0.0,1.0,0.0,0.0,0.000000,2.4,0.0


In [28]:
corr_matrix = X_train.corr().abs()

upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

threshold = 0.4

to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > threshold)]

to_drop

['ckd',
 'creatinine',
 'side_stenosis',
 'side_predilat',
 'minor_criteria',
 'valvular_disease',
 'side_diametr',
 'major_lm']

In [29]:
X_train = X_train.drop(columns=to_drop)
X_test = X_test.drop(columns=to_drop)
X_val = X_val.drop(columns=to_drop)

X_train

,age,anemia,ef,cerebrovascular_disease,peripheral_artery_disease,if_yes_what_type___1,single_vessel,calcium,stent_type___3,medina_side,...,def,history_of_cancer,stent_type___5,previous_stroke_tia,stent_diameter,stent_length,adhoc_pci,previous_pci,stent_type___4,cto_bifurc
0,41.0,0.0,49.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.477395,4.00,11.0,0.0,1.0,0.0,0.0
1,56.0,0.0,68.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.313995,2.75,18.0,1.0,1.0,1.0,0.0
2,55.0,0.0,70.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.478905,2.75,33.0,1.0,0.0,1.0,0.0
3,79.0,0.0,73.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,3.50,38.0,0.0,0.0,0.0,0.0
4,61.0,0.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.448962,4.00,28.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,69.0,0.0,60.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.525775,3.50,32.0,0.0,1.0,0.0,0.0
1222,57.0,0.0,39.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.616766,3.50,23.0,0.0,0.0,0.0,0.0
1223,63.0,1.0,71.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.429977,2.75,18.0,0.0,1.0,0.0,0.0
1224,75.0,0.0,69.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.000000,3.00,18.0,0.0,1.0,0.0,0.0


In [30]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(1226, 22)
(409, 22)
(409, 22)


In [31]:
print(X_train.columns)

Index(['age', 'anemia', 'ef', 'cerebrovascular_disease',
       'peripheral_artery_disease', 'if_yes_what_type___1', 'single_vessel',
       'calcium', 'stent_type___3', 'medina_side', 'atrial_fibrilation',
       'height', 'def', 'history_of_cancer', 'stent_type___5',
       'previous_stroke_tia', 'stent_diameter', 'stent_length', 'adhoc_pci',
       'previous_pci', 'stent_type___4', 'cto_bifurc'],
      dtype='object')


# Syntentic data

In [32]:
X_train_with_ID = X_train.copy().reset_index(drop=False)
X_train_with_ID

,index,age,anemia,ef,cerebrovascular_disease,peripheral_artery_disease,if_yes_what_type___1,single_vessel,calcium,stent_type___3,...,def,history_of_cancer,stent_type___5,previous_stroke_tia,stent_diameter,stent_length,adhoc_pci,previous_pci,stent_type___4,cto_bifurc
0,0,41.0,0.0,49.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.477395,4.00,11.0,0.0,1.0,0.0,0.0
1,1,56.0,0.0,68.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.313995,2.75,18.0,1.0,1.0,1.0,0.0
2,2,55.0,0.0,70.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.478905,2.75,33.0,1.0,0.0,1.0,0.0
3,3,79.0,0.0,73.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.000000,3.50,38.0,0.0,0.0,0.0,0.0
4,4,61.0,0.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.448962,4.00,28.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,1221,69.0,0.0,60.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.525775,3.50,32.0,0.0,1.0,0.0,0.0
1222,1222,57.0,0.0,39.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.616766,3.50,23.0,0.0,0.0,0.0,0.0
1223,1223,63.0,1.0,71.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.429977,2.75,18.0,0.0,1.0,0.0,0.0
1224,1224,75.0,0.0,69.0,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,1.000000,3.00,18.0,0.0,1.0,0.0,0.0


In [33]:
X_train_with_ID_and_target = X_train_with_ID.copy()
X_train_with_ID_and_target['target'] = y_train
X_train_with_ID_and_target

,index,age,anemia,ef,cerebrovascular_disease,peripheral_artery_disease,if_yes_what_type___1,single_vessel,calcium,stent_type___3,...,history_of_cancer,stent_type___5,previous_stroke_tia,stent_diameter,stent_length,adhoc_pci,previous_pci,stent_type___4,cto_bifurc,target
0,0,41.0,0.0,49.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.477395,4.00,11.0,0.0,1.0,0.0,0.0,0
1,1,56.0,0.0,68.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.313995,2.75,18.0,1.0,1.0,1.0,0.0,0
2,2,55.0,0.0,70.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.478905,2.75,33.0,1.0,0.0,1.0,0.0,0
3,3,79.0,0.0,73.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.000000,3.50,38.0,0.0,0.0,0.0,0.0,0
4,4,61.0,0.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.448962,4.00,28.0,1.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,1221,69.0,0.0,60.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.525775,3.50,32.0,0.0,1.0,0.0,0.0,1
1222,1222,57.0,0.0,39.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.616766,3.50,23.0,0.0,0.0,0.0,0.0,0
1223,1223,63.0,1.0,71.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.429977,2.75,18.0,0.0,1.0,0.0,0.0,0
1224,1224,75.0,0.0,69.0,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.000000,3.00,18.0,0.0,1.0,0.0,0.0,0


In [34]:
from sdv.metadata import Metadata

metadata = Metadata.detect_from_dataframe(data=X_train_with_ID_and_target)

In [35]:
from sdv.single_table import GaussianCopulaSynthesizer, CTGANSynthesizer, TVAESynthesizer, CopulaGANSynthesizer

# synthesizer = GaussianCopulaSynthesizer(metadata, default_distribution='norm')
synthesizer = CTGANSynthesizer(metadata, epochs=500)
# synthesizer = TVAESynthesizer(metadata, epochs=500)
# synthesizer = CopulaGANSynthesizer(metadata, epochs=500, default_distribution='norm')
synthesizer.fit(X_train_with_ID_and_target)

synthetic_data = synthesizer.sample(num_rows=10000)
synthetic_data

/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sdv/single_table/base.py:104: UserWarning:

We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.



,index,age,anemia,ef,cerebrovascular_disease,peripheral_artery_disease,if_yes_what_type___1,single_vessel,calcium,stent_type___3,...,history_of_cancer,stent_type___5,previous_stroke_tia,stent_diameter,stent_length,adhoc_pci,previous_pci,stent_type___4,cto_bifurc,target
0,12053904,47.0,0.0,50.346024,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.657644,4.062913,14.193122,0.0,1.0,0.0,1.0,0
1,13010658,57.0,0.0,56.943700,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.479666,2.923050,23.748550,1.0,0.0,0.0,0.0,1
2,9912584,61.0,0.0,59.172447,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.539464,3.057108,23.632414,0.0,0.0,0.0,0.0,0
3,2797363,33.0,1.0,67.164850,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.653522,2.790355,20.465990,0.0,0.0,0.0,0.0,0
4,13661511,62.0,0.0,55.098254,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.536789,3.517832,13.584815,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,4519439,28.0,0.0,52.740856,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.669708,3.022155,16.199966,0.0,0.0,0.0,0.0,0
9996,657099,48.0,0.0,54.827055,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.404172,3.504870,36.681735,0.0,1.0,0.0,0.0,0
9997,3822342,50.0,0.0,59.286793,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.399949,4.167753,36.678036,1.0,0.0,0.0,0.0,0
9998,2301487,44.0,0.0,35.555135,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.461308,3.894040,15.664263,1.0,0.0,0.0,1.0,0


In [36]:
unique, counts = np.unique(synthetic_data['target'], return_counts=True)
value_counts = dict(zip(unique, counts))
value_counts

{0: 9269, 1: 731}

In [37]:
minority_synthetic_data = synthetic_data[synthetic_data['target'] == 1]
minority_synthetic_data

,index,age,anemia,ef,cerebrovascular_disease,peripheral_artery_disease,if_yes_what_type___1,single_vessel,calcium,stent_type___3,...,history_of_cancer,stent_type___5,previous_stroke_tia,stent_diameter,stent_length,adhoc_pci,previous_pci,stent_type___4,cto_bifurc,target
1,13010658,57.0,0.0,56.943700,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.479666,2.923050,23.748550,1.0,0.0,0.0,0.0,1
21,2160786,58.0,0.0,49.675994,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.434316,3.008264,22.696111,0.0,0.0,0.0,0.0,1
66,12283918,68.0,0.0,70.286009,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.289346,2.805997,21.986228,0.0,0.0,0.0,0.0,1
74,13644400,72.0,0.0,55.225442,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.366731,3.599666,21.368627,0.0,1.0,0.0,0.0,1
86,7801502,45.0,0.0,48.304705,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.390479,3.120116,10.535710,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9940,12360188,60.0,0.0,60.186084,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.067683,3.093169,13.315455,1.0,1.0,0.0,0.0,1
9977,2885332,38.0,0.0,75.532074,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.481619,4.349811,16.323759,1.0,0.0,0.0,0.0,1
9979,3851326,45.0,0.0,59.303131,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.468077,3.099331,9.479426,0.0,0.0,0.0,0.0,1
9980,3972568,60.0,0.0,17.000000,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.548140,2.462233,16.797823,0.0,1.0,0.0,1.0,1


In [38]:
syntetic_minority_dropped = minority_synthetic_data.copy().drop(['target', 'index'], axis=1)
syntetic_minority_dropped

,age,anemia,ef,cerebrovascular_disease,peripheral_artery_disease,if_yes_what_type___1,single_vessel,calcium,stent_type___3,medina_side,...,def,history_of_cancer,stent_type___5,previous_stroke_tia,stent_diameter,stent_length,adhoc_pci,previous_pci,stent_type___4,cto_bifurc
1,57.0,0.0,56.943700,0.0,0.0,1.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.479666,2.923050,23.748550,1.0,0.0,0.0,0.0
21,58.0,0.0,49.675994,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.434316,3.008264,22.696111,0.0,0.0,0.0,0.0
66,68.0,0.0,70.286009,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.289346,2.805997,21.986228,0.0,0.0,0.0,0.0
74,72.0,0.0,55.225442,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.366731,3.599666,21.368627,0.0,1.0,0.0,0.0
86,45.0,0.0,48.304705,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.390479,3.120116,10.535710,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9940,60.0,0.0,60.186084,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.067683,3.093169,13.315455,1.0,1.0,0.0,0.0
9977,38.0,0.0,75.532074,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.481619,4.349811,16.323759,1.0,0.0,0.0,0.0
9979,45.0,0.0,59.303131,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.468077,3.099331,9.479426,0.0,0.0,0.0,0.0
9980,60.0,0.0,17.000000,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.548140,2.462233,16.797823,0.0,1.0,0.0,1.0


In [39]:
syntetic_target = minority_synthetic_data['target']
syntetic_target

1       1
21      1
66      1
74      1
86      1
       ..
9940    1
9977    1
9979    1
9980    1
9981    1
Name: target, Length: 731, dtype: int64

In [40]:
X_train_new = pd.concat([X_train, syntetic_minority_dropped])
X_train_new

,age,anemia,ef,cerebrovascular_disease,peripheral_artery_disease,if_yes_what_type___1,single_vessel,calcium,stent_type___3,medina_side,...,def,history_of_cancer,stent_type___5,previous_stroke_tia,stent_diameter,stent_length,adhoc_pci,previous_pci,stent_type___4,cto_bifurc
0,41.0,0.0,49.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.477395,4.000000,11.000000,0.0,1.0,0.0,0.0
1,56.0,0.0,68.000000,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.313995,2.750000,18.000000,1.0,1.0,1.0,0.0
2,55.0,0.0,70.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.478905,2.750000,33.000000,1.0,0.0,1.0,0.0
3,79.0,0.0,73.000000,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,3.500000,38.000000,0.0,0.0,0.0,0.0
4,61.0,0.0,62.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.448962,4.000000,28.000000,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9940,60.0,0.0,60.186084,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.067683,3.093169,13.315455,1.0,1.0,0.0,0.0
9977,38.0,0.0,75.532074,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.481619,4.349811,16.323759,1.0,0.0,0.0,0.0
9979,45.0,0.0,59.303131,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.468077,3.099331,9.479426,0.0,0.0,0.0,0.0
9980,60.0,0.0,17.000000,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.548140,2.462233,16.797823,0.0,1.0,0.0,1.0


In [41]:
y_train_new = np.concatenate((y_train, syntetic_target))
y_train_new

array([0, 0, 0, ..., 1, 1, 1])

# Оптимизация logistic regression на auc-roc для 10 фолдов

In [42]:
print('LogReg for classification binary 10 folds')

LogReg for classification binary 10 folds


In [43]:
from sklearn.preprocessing import StandardScaler

scaller = StandardScaler()
scaller.fit(X_train)

StandardScaler()

In [44]:
import joblib

joblib.dump(scaller, "./scaler.save")

['./scaler.save']

In [45]:
X_train_folds = pd.concat([X_train, X_val])
y_train_k_fold = np.concatenate((y_train, y_val), axis=0)

In [46]:
scaled_features = scaller.transform(X_train_folds)

In [47]:
from imblearn.over_sampling import SMOTE
from collections import Counter

def resampled_with_smote(X_train, y_train, resamble_neg, resamble_pos):
    sampling_strategy = {0: int(np.bincount(y_train)[0] * resamble_neg), 1: int(np.bincount(y_train)[1] * resamble_pos)}
    smote = SMOTE(sampling_strategy=sampling_strategy, random_state=42)

    # Генерация новых примеров
    X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
    return X_resampled, y_resampled

In [48]:
from joblib import dump, load

# smote

In [52]:
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
from sklearn.linear_model import LogisticRegression
import numpy as np
import json
from joblib import dump
from collections import Counter

def objective(params):
    # Convert parameters to proper format
    params['C'] = float(params['C'])
    params['max_iter'] = int(params['max_iter'])
    
    classifier = LogisticRegression(
        C=params['C'],
        max_iter=int(params['max_iter']),
        solver=params['solver'],
        verbose=0,
        n_jobs=-1
    )
    
    kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    roc_auc_scores = []
    f1_scores = []
    precision_scores = []
    recall_scores = []
    accuracy_scores = []

    for train_index, test_index in kf.split(X_train_folds, y_train_k_fold):
        not_scaled_X_train_folds = X_train_folds.copy()
        scaled_X_train_folds = scaller.transform(not_scaled_X_train_folds)
        X_test = scaled_X_train_folds[test_index]
        y_test = y_train_k_fold[test_index]

        # Get original training data
        X_train = scaled_X_train_folds[train_index]
        y_train = y_train_k_fold[train_index]
        
        # Apply SMOTE oversampling - keep original negative samples and increase positive ones
        X_train_resampled, y_train_resampled = resampled_with_smote(
            X_train, y_train, 
            resamble_neg=1.0,  # Keep all negative samples
            resamble_pos=3.0   # Triple the positive samples (deaths)
        )
        
        # Print class distribution before and after resampling
        print(f"Original class distribution: {Counter(y_train)}")
        print(f"Resampled class distribution: {Counter(y_train_resampled)}")
        
        classifier.fit(X_train_resampled, y_train_resampled)
        predictions = classifier.predict_proba(X_test)[:, 1]
        y_pred = classifier.predict(X_test)
        
        roc_auc = roc_auc_score(y_test, predictions)
        f1 = f1_score(y_test, y_pred, zero_division=0)
        precision = precision_score(y_test, y_pred, zero_division=0)
        recall = recall_score(y_test, y_pred, zero_division=0)
        accuracy = accuracy_score(y_test, y_pred)
        
        roc_auc_scores.append(roc_auc)
        f1_scores.append(f1)
        precision_scores.append(precision)
        recall_scores.append(recall)
        accuracy_scores.append(accuracy)

    mean_metrics = {
        'roc_auc': np.mean(roc_auc_scores),
        'f1': np.mean(f1_scores),
        'precision': np.mean(precision_scores),
        'recall': np.mean(recall_scores),
        'accuracy': np.mean(accuracy_scores),
        'roc_auc_scores': roc_auc_scores,
        'f1_scores': f1_scores,
        'precision_scores': precision_scores,
        'recall_scores': recall_scores,
        'accuracy_scores': accuracy_scores
    }
    
    return {'loss': -mean_metrics['roc_auc'], 'status': STATUS_OK, 'params': params, 'mean_metrics': mean_metrics}

# Define the search space
space = {
    'C': hp.loguniform('C', np.log(0.001), np.log(10)),
    'max_iter': hp.quniform('max_iter', 100, 1000, 100),
    'solver': hp.choice('solver', ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'])
}

trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10,
            trials=trials)

# Extracting metrics and params from the best trial
best_trial = trials.best_trial['result']
best_params = best_trial['params']
best_metrics = best_trial['mean_metrics']

# Train the model with the best hyperparameters on the full training set with SMOTE
best_model = LogisticRegression(**best_params, verbose=0)

# Apply SMOTE to the full training set
X_train_scaled = scaller.transform(X_train_folds)
X_train_resampled, y_train_resampled = resampled_with_smote(
    X_train_scaled, y_train_k_fold, 
    resamble_neg=1.0,  # Keep all negative samples
    resamble_pos=3.0   # Triple the positive samples (deaths)
)

# Print class distribution before and after resampling for full training set
print(f"Full training set - Original class distribution: {Counter(y_train_k_fold)}")
print(f"Full training set - Resampled class distribution: {Counter(y_train_resampled)}")

# Train on the resampled data
best_model.fit(X_train_resampled, y_train_resampled)

# Save the best model and metrics
model_filename = 'LogReg_SMOTE_CV.joblib'
dump(best_model, model_filename)

# Evaluate best model on test dataset
# Scale the test data using the same scaler
X_test_scaled = scaller.transform(X_test)

# Make predictions
test_predictions_proba = best_model.predict_proba(X_test_scaled)[:, 1]
test_predictions = best_model.predict(X_test_scaled)

# Calculate metrics on test data
test_metrics = {
    'roc_auc': roc_auc_score(y_test, test_predictions_proba),
    'f1': f1_score(y_test, test_predictions, zero_division=0),
    'precision': precision_score(y_test, test_predictions, zero_division=0),
    'recall': recall_score(y_test, test_predictions, zero_division=0),
    'accuracy': accuracy_score(y_test, test_predictions)
}

print("Test set evaluation:")
for metric, value in test_metrics.items():
    print(f"{metric}: {value:.4f}")

# Save metrics and best parameters to a file
metrics_and_params = {
    'best_parameters': best_params,
    'cross_validation_metrics': best_metrics,
    'test_metrics': test_metrics
}

with open('scores_LogReg_SMOTE_CV.json', 'w') as f:
    json.dump(metrics_and_params, f)

print("Best parameters, model, and evaluation metrics saved together.")

Original class distribution: Counter({0: 1305, 1: 106})
Resampled class distribution: Counter({0: 1305, 1: 318})
Original class distribution: Counter({0: 1306, 1: 105})
Resampled class distribution: Counter({0: 1306, 1: 315})
Original class distribution: Counter({0: 1306, 1: 105})
Resampled class distribution: Counter({0: 1306, 1: 315})
Original class distribution: Counter({0: 1306, 1: 105})
Resampled class distribution: Counter({0: 1306, 1: 315})
Original class distribution: Counter({0: 1306, 1: 105})
Resampled class distribution: Counter({0: 1306, 1: 315})
Original class distribution: Counter({0: 1306, 1: 105})
Resampled class distribution: Counter({0: 1306, 1: 315})
Original class distribution: Counter({0: 1306, 1: 105})
Resampled class distribution: Counter({0: 1306, 1: 315})
Original class distribution: Counter({0: 1306, 1: 105})
Resampled class distribution: Counter({0: 1306, 1: 315})
Original class distribution: Counter({0: 1306, 1: 106})
Resampled class distribution: Counter({0

# no smote

In [51]:
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
from sklearn.linear_model import LogisticRegression
import numpy as np
import json
from joblib import dump

def objective(params):
    # Convert parameters to proper format
    params['C'] = float(params['C'])
    params['max_iter'] = int(params['max_iter'])
    
    classifier = LogisticRegression(
        C=params['C'],
        max_iter=int(params['max_iter']),
        solver=params['solver'],
        verbose=0,
        n_jobs=-1
    )
    
    kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    roc_auc_scores = []
    f1_scores = []
    precision_scores = []
    recall_scores = []
    accuracy_scores = []

    for train_index, test_index in kf.split(X_train_folds, y_train_k_fold):
        not_scaled_X_train_folds = X_train_folds.copy()
        scaled_X_train_folds = scaller.transform(not_scaled_X_train_folds)
        X_test = scaled_X_train_folds[test_index]
        y_test = y_train_k_fold[test_index]

        # Use original training data without synthetic augmentation
        X_train = scaled_X_train_folds[train_index]
        y_train = y_train_k_fold[train_index]
        
        classifier.fit(X_train, y_train)
        predictions = classifier.predict_proba(X_test)[:, 1]
        y_pred = classifier.predict(X_test)
        
        roc_auc = roc_auc_score(y_test, predictions)
        f1 = f1_score(y_test, y_pred, zero_division=0)
        precision = precision_score(y_test, y_pred, zero_division=0)
        recall = recall_score(y_test, y_pred, zero_division=0)
        accuracy = accuracy_score(y_test, y_pred)
        
        roc_auc_scores.append(roc_auc)
        f1_scores.append(f1)
        precision_scores.append(precision)
        recall_scores.append(recall)
        accuracy_scores.append(accuracy)

    mean_metrics = {
        'roc_auc': np.mean(roc_auc_scores),
        'f1': np.mean(f1_scores),
        'precision': np.mean(precision_scores),
        'recall': np.mean(recall_scores),
        'accuracy': np.mean(accuracy_scores),
        'roc_auc_scores': roc_auc_scores,
        'f1_scores': f1_scores,
        'precision_scores': precision_scores,
        'recall_scores': recall_scores,
        'accuracy_scores': accuracy_scores
    }
    
    return {'loss': -mean_metrics['roc_auc'], 'status': STATUS_OK, 'params': params, 'mean_metrics': mean_metrics}

# Define the search space
space = {
    'C': hp.loguniform('C', np.log(0.001), np.log(10)),
    'max_iter': hp.quniform('max_iter', 100, 1000, 100),
    'solver': hp.choice('solver', ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'])
}

trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=100,
            trials=trials)

# Extracting metrics and params from the best trial
best_trial = trials.best_trial['result']
best_params = best_trial['params']
best_metrics = best_trial['mean_metrics']

# Train the model with the best hyperparameters
best_model = LogisticRegression(**best_params, verbose=0)
best_model.fit(X_train_folds, y_train_k_fold)

# Save the best model and metrics
model_filename = 'LogReg_CV.joblib'
dump(best_model, model_filename)

# Evaluate best model on test dataset
# Scale the test data using the same scaler
X_test_scaled = scaller.transform(X_test)

# Make predictions
test_predictions_proba = best_model.predict_proba(X_test_scaled)[:, 1]
test_predictions = best_model.predict(X_test_scaled)

# Calculate metrics on test data
test_metrics = {
    'roc_auc': roc_auc_score(y_test, test_predictions_proba),
    'f1': f1_score(y_test, test_predictions, zero_division=0),
    'precision': precision_score(y_test, test_predictions, zero_division=0),
    'recall': recall_score(y_test, test_predictions, zero_division=0),
    'accuracy': accuracy_score(y_test, test_predictions)
}

print("Test set evaluation:")
for metric, value in test_metrics.items():
    print(f"{metric}: {value:.4f}")

# Save metrics and best parameters to a file
metrics_and_params = {
    'best_parameters': best_params,
    'cross_validation_metrics': best_metrics,
    'test_metrics': test_metrics
}

with open('scores_LogReg_CV.json', 'w') as f:
    json.dump(metrics_and_params, f)

print("Best parameters, model, and evaluation metrics saved together.")

  5%|▌         | 5/100 [00:00<00:07, 12.11trial/s, best loss: -0.7590972788823521]

/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 10.


/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 10.


/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 10.


/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 10.


/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is s

 18%|█▊        | 18/100 [00:02<00:14,  5.61trial/s, best loss: -0.7590972788823521]

/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 10.


/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 10.


/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 10.


/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 10.


/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is s

 34%|███▍      | 34/100 [00:05<00:10,  6.02trial/s, best loss: -0.7593789811196519]

...

/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 10.


/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 10.


/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 10.


/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 10.


/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is s

 92%|█████████▏| 92/100 [00:14<00:01,  6.28trial/s, best loss: -0.7593837763559068]

/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 10.


/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 10.


/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 10.


/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 10.


/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is s

100%|██████████| 100/100 [00:15<00:00,  6.34trial/s, best loss: -0.7593837763559068]
Test set evaluation:
roc_auc: 0.6877
f1: 0.0000
precision: 0.0000
recall: 0.0000
accuracy: 0.9262
Best parameters, model, and evaluation metrics saved together.


/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning:

X does not have valid feature names, but LogisticRegression was fitted with feature names

/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning:

X does not have valid feature names, but LogisticRegression was fitted with feature names



# LOGREG + GAN + CV

In [53]:
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
from sklearn.linear_model import LogisticRegression
import numpy as np
import json

def objective(params):
    # Convert parameters to proper format
    params['C'] = float(params['C'])
    params['max_iter'] = int(params['max_iter'])
    
    classifier = LogisticRegression(
        C=params['C'],
        max_iter=int(params['max_iter']),
        solver=params['solver'],
        verbose=0,
        n_jobs=-1
    )
    
    kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    roc_auc_scores = []
    f1_scores = []
    precision_scores = []
    recall_scores = []
    accuracy_scores = []

    for train_index, test_index in kf.split(X_train_folds, y_train_k_fold):
        not_scaled_X_train_folds = X_train_folds.copy()
        scaled_X_train_folds = scaller.transform(not_scaled_X_train_folds)
        X_test = scaled_X_train_folds[test_index]
        y_test = y_train_k_fold[test_index]

        train = not_scaled_X_train_folds.iloc[train_index].copy().reset_index(drop=True)
        train['index'] = np.arange(1, len(train) + 1)
        train['target'] = y_train_k_fold[train_index]
        synthesizer = CTGANSynthesizer(metadata, epochs=500)
        synthesizer.fit(train)
        synthetic_data = synthesizer.sample(num_rows=10000)
        minority_synthetic_data = synthetic_data[synthetic_data['target'] == 1]
        if len(minority_synthetic_data) > 600:
            minority_synthetic_data = minority_synthetic_data.sample(n=600)
        syntetic_minority_dropped = minority_synthetic_data.copy().drop(['target', 'index'], axis=1)
        syntetic_target = minority_synthetic_data['target']

        X_train = pd.concat([not_scaled_X_train_folds.iloc[train_index], syntetic_minority_dropped])
        X_train = scaller.transform(X_train)
        y_train = np.concatenate((y_train_k_fold[train_index], syntetic_target), axis=0)
        
        classifier.fit(X_train, y_train)
        predictions = classifier.predict_proba(X_test)[:, 1]
        y_pred = classifier.predict(X_test)
        
        roc_auc = roc_auc_score(y_test, predictions)
        f1 = f1_score(y_test, y_pred, zero_division=0)
        precision = precision_score(y_test, y_pred, zero_division=0)
        recall = recall_score(y_test, y_pred, zero_division=0)
        accuracy = accuracy_score(y_test, y_pred)
        
        roc_auc_scores.append(roc_auc)
        f1_scores.append(f1)
        precision_scores.append(precision)
        recall_scores.append(recall)
        accuracy_scores.append(accuracy)

    mean_metrics = {
        'roc_auc': np.mean(roc_auc_scores),
        'f1': np.mean(f1_scores),
        'precision': np.mean(precision_scores),
        'recall': np.mean(recall_scores),
        'accuracy': np.mean(accuracy_scores),
        'roc_auc_scores': roc_auc_scores,
        'f1_scores': f1_scores,
        'precision_scores': precision_scores,
        'recall_scores': recall_scores,
        'accuracy_scores': accuracy_scores
    }
    
    return {'loss': -mean_metrics['roc_auc'], 'status': STATUS_OK, 'params': params, 'mean_metrics': mean_metrics}

# Define the search space
space = {
    'C': hp.loguniform('C', np.log(0.001), np.log(10)),
    'max_iter': hp.quniform('max_iter', 100, 1000, 100),
    'solver': hp.choice('solver', ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'])
}

trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10,
            trials=trials)

# Extracting metrics and params from the best trial
best_trial = trials.best_trial['result']
best_params = best_trial['params']
best_metrics = best_trial['mean_metrics']

# Train the model with the best hyperparameters
best_model = LogisticRegression(**best_params, verbose=0)
best_model.fit(X_train_folds, y_train_k_fold)

# Save the best model and metrics
# Assuming X_train_folds and y_train_k_fold are defined elsewhere in your code

model_filename = 'LogReg_GAN_CV.joblib'
dump(best_model, model_filename)

# Save metrics and best parameters to a file
metrics_and_params = {
    'best_parameters': best_params,
    'evaluation_metrics': best_metrics
}

with open('scores_LogReg_GAN_CV.json', 'w') as f:
    json.dump(metrics_and_params, f)

print("Best parameters, model, and evaluation metrics saved together.")

# Evaluate the best model on test dataset
print("Evaluating best model on test dataset...")

# Make sure to apply the same preprocessing to test data as was done for training data
X_test_scaled = scaller.transform(X_test)  # Assuming X_test is your test dataset

# Make predictions
test_predictions = best_model.predict_proba(X_test_scaled)[:, 1]
test_pred_labels = best_model.predict(X_test_scaled)

# Calculate evaluation metrics
test_roc_auc = roc_auc_score(y_test, test_predictions)
test_f1 = f1_score(y_test, test_pred_labels, zero_division=0)
test_precision = precision_score(y_test, test_pred_labels, zero_division=0)
test_recall = recall_score(y_test, test_pred_labels, zero_division=0)
test_accuracy = accuracy_score(y_test, test_pred_labels)

# Create dictionary for test metrics
test_metrics = {
    'test_roc_auc': float(test_roc_auc),
    'test_f1': float(test_f1),
    'test_precision': float(test_precision),
    'test_recall': float(test_recall),
    'test_accuracy': float(test_accuracy)
}

print("Test Metrics:")
for metric_name, metric_value in test_metrics.items():
    print(f"{metric_name}: {metric_value:.4f}")

# Add test metrics to the saved JSON file
metrics_and_params['test_metrics'] = test_metrics

# Save updated metrics including test results
with open('scores_LogReg_GAN_CV.json', 'w') as f:
    json.dump(metrics_and_params, f, indent=4)

print("Model evaluation on test dataset completed and results saved.")

 20%|██        | 2/10 [14:09<56:28, 423.61s/trial, best loss: -0.6092143685317989]  

/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 10.


/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 10.


/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 10.


/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 10.


/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is s

 30%|███       | 3/10 [37:08<1:40:20, 860.07s/trial, best loss: -0.6092143685317989]

/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 10.




 30%|███       | 3/10 [44:59<1:44:58, 899.75s/trial, best loss: -0.6092143685317989]


KeyboardInterrupt: 

# LOGREG + GAN

In [53]:
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc
from sklearn.linear_model import LogisticRegression

# Define the best parameters for LogisticRegression (you would normally get these from hyperparameter tuning)
# These are just examples - replace with your actual best parameters
best_params = {
  "C": 0.07416188374104146,
  "max_iter": 200,
  "solver": "sag"
}
# Train the model with the best hyperparameters on X_train_folds
if not isinstance(X_train_folds, pd.DataFrame):
    X_train_df = pd.DataFrame(X_train_folds)
else:
    X_train_df = X_train_folds.copy()

# Add target column to the training data
X_train_df['target'] = y_train_k_fold
X_train_df['index'] = np.arange(1, len(X_train_df) + 1)

# Generate synthetic data using CTGAN
synthesizer = CTGANSynthesizer(metadata, epochs=500)
synthesizer.fit(X_train_df)
synthetic_data = synthesizer.sample(num_rows=10000)
minority_synthetic_data = synthetic_data[synthetic_data['target'] == 1]
if len(minority_synthetic_data) > 600:
    minority_synthetic_data = minority_synthetic_data.sample(n=600)

# Extract features and target from synthetic data
syntetic_minority_dropped = minority_synthetic_data.copy().drop(['target', 'index'], axis=1)
syntetic_target = minority_synthetic_data['target']

# Combine original and synthetic data
X_train_combined = pd.concat([X_train_df.drop(['target', 'index'], axis=1), syntetic_minority_dropped])
y_train_combined = np.concatenate((y_train_k_fold, syntetic_target), axis=0)

# Scale the combined training data
X_train_combined_scaled = scaller.fit_transform(X_train_combined)

# Train the model with the best hyperparameters on scaled combined data
best_model = LogisticRegression(**best_params, verbose=0, n_jobs=-1)
best_model.fit(X_train_combined_scaled, y_train_combined)

# Scale the test data
X_test_scaled = scaller.transform(X_test)

# Make predictions
test_predictions = best_model.predict_proba(X_test_scaled)[:, 1]
test_pred_labels = best_model.predict(X_test_scaled)

# Calculate evaluation metrics
test_roc_auc = roc_auc_score(y_test, test_predictions)
test_f1 = f1_score(y_test, test_pred_labels, zero_division=0)
test_precision = precision_score(y_test, test_pred_labels, zero_division=0)
test_recall = recall_score(y_test, test_pred_labels, zero_division=0)
test_accuracy = accuracy_score(y_test, test_pred_labels)

# Create dictionary for test metrics
test_metrics = {
    'test_roc_auc': float(test_roc_auc),
    'test_f1': float(test_f1),
    'test_precision': float(test_precision),
    'test_recall': float(test_recall),
    'test_accuracy': float(test_accuracy)
}

# Print the metrics
print("Test Metrics:")
for metric_name, metric_value in test_metrics.items():
    print(f"{metric_name}: {metric_value:.4f}")

# Save the model and metrics
from joblib import dump
model_filename = 'LogReg_GAN_test.joblib'
dump(best_model, model_filename)

# Save the test metrics to a file
metrics_and_params = {
    'best_parameters': best_params,
    'test_metrics': test_metrics
}

with open('test_scores_LogReg_GAN.json', 'w') as f:
    json.dump(metrics_and_params, f, indent=4)



Test Metrics:
test_roc_auc: 0.5850
test_f1: 0.2090
test_precision: 0.1944
test_recall: 0.2258
test_accuracy: 0.8704


# LOGREG + ARF

In [70]:
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc
from sklearn.linear_model import LogisticRegression
import synthcity.logger as log
from synthcity.plugins import Plugins
from synthcity.plugins.core.dataloader import GenericDataLoader

# Prepare the data for ARF
train_df = X_train_folds.copy() if isinstance(X_train_folds, pd.DataFrame) else pd.DataFrame(X_train_folds)
train_df['target'] = y_train_k_fold

# Create a data loader for synthcity
loader = GenericDataLoader(
    train_df,
    target_column="target",
)

# Initialize and fit the ARF model
syn_model = Plugins().get("arf")
syn_model.fit(loader)

# Generate synthetic data
synthetic_data = syn_model.generate(count=10000).dataframe()

# Extract minority class samples
minority_synthetic_data = synthetic_data[synthetic_data['target'] == 1]
if len(minority_synthetic_data) > 600:
    minority_synthetic_data = minority_synthetic_data.sample(n=600)

# Extract features and target from synthetic data
syntetic_minority_features = minority_synthetic_data.drop('target', axis=1)
syntetic_target = minority_synthetic_data['target']

# Combine original and synthetic data
X_train_combined = pd.concat([train_df.drop('target', axis=1), syntetic_minority_features])
y_train_combined = np.concatenate((y_train_k_fold, syntetic_target.values), axis=0)

# Scale the combined training data
X_train_combined_scaled = scaller.fit_transform(X_train_combined)

# Train the model with the best hyperparameters on scaled combined data
best_model = LogisticRegression(**best_params, verbose=0, n_jobs=-1)
best_model.fit(X_train_combined_scaled, y_train_combined)

# Scale the test data
X_test_scaled = scaller.transform(X_test)

# Make predictions
test_predictions = best_model.predict_proba(X_test_scaled)[:, 1]
test_pred_labels = best_model.predict(X_test_scaled)

# Calculate evaluation metrics
test_roc_auc = roc_auc_score(y_test, test_predictions)
test_f1 = f1_score(y_test, test_pred_labels, zero_division=0)
test_precision = precision_score(y_test, test_pred_labels, zero_division=0)
test_recall = recall_score(y_test, test_pred_labels, zero_division=0)
test_accuracy = accuracy_score(y_test, test_pred_labels)

# Create dictionary for test metrics
test_metrics = {
    'test_roc_auc': float(test_roc_auc),
    'test_f1': float(test_f1),
    'test_precision': float(test_precision),
    'test_recall': float(test_recall),
    'test_accuracy': float(test_accuracy)
}

# Print the metrics
print("Test Metrics:")
for metric_name, metric_value in test_metrics.items():
    print(f"{metric_name}: {metric_value:.4f}")

# Save the model and metrics
from joblib import dump
model_filename = 'LogReg_ARF.joblib'
dump(best_model, model_filename)

# Save the test metrics to a file
metrics_and_params = {
    'best_parameters': best_params,
    'test_metrics': test_metrics
}

with open('test_scores_LogReg_ARF.json', 'w') as f:
    json.dump(metrics_and_params, f, indent=4)

print("Model trained on ARF-augmented data, evaluated on test dataset, and results saved.")

[2025-05-26T17:24:14.450199+0400][15250][CRITICAL] load failed: module 'synthcity.plugins.generic.plugin_great' has no attribute 'plugin'
[2025-05-26T17:24:14.451314+0400][15250][CRITICAL] load failed: module 'synthcity.plugins.generic.plugin_great' has no attribute 'plugin'
[2025-05-26T17:24:14.451918+0400][15250][CRITICAL] module plugin_great load failed
[2025-05-26T17:24:14.452400+0400][15250][CRITICAL] module disabled: /Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/synthcity/plugins/generic/plugin_goggle.py


Initial accuracy is 0.6431192660550459
Iteration number 1 reached accuracy of 0.44067278287461775.
Test Metrics:
test_roc_auc: 0.7804
test_f1: 0.3684
test_precision: 0.3111
test_recall: 0.4516
test_accuracy: 0.8826
Model trained on ARF-augmented data, evaluated on test dataset, and results saved.


In [ ]:
Test Metrics:
test_roc_auc: 0.7804
test_f1: 0.3684
test_precision: 0.3111
test_recall: 0.4516
test_accuracy: 0.8826


# LOGREG + TVAE

In [61]:
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc
from sklearn.linear_model import LogisticRegression

# Define the best parameters for LogisticRegression (you would normally get these from hyperparameter tuning)
# These are just examples - replace with your actual best parameters
best_params = {
  "C": 0.07416188374104146,
  "max_iter": 200,
  "solver": "sag"
}
# Train the model with the best hyperparameters on X_train_folds
if not isinstance(X_train_folds, pd.DataFrame):
    X_train_df = pd.DataFrame(X_train_folds)
else:
    X_train_df = X_train_folds.copy()

# Add target column to the training data
X_train_df['target'] = y_train_k_fold
X_train_df['index'] = np.arange(1, len(X_train_df) + 1)

# Generate synthetic data using TVAE
synthesizer = TVAESynthesizer(metadata)
synthesizer.fit(X_train_df)
synthetic_data = synthesizer.sample(num_rows=10000)
minority_synthetic_data = synthetic_data[synthetic_data['target'] == 1]
if len(minority_synthetic_data) > 600:
    minority_synthetic_data = minority_synthetic_data.sample(n=600)

# Extract features and target from synthetic data
syntetic_minority_dropped = minority_synthetic_data.copy().drop(['target', 'index'], axis=1)
syntetic_target = minority_synthetic_data['target']

# Combine original and synthetic data
X_train_combined = pd.concat([X_train_df.drop(['target', 'index'], axis=1), syntetic_minority_dropped])
y_train_combined = np.concatenate((y_train_k_fold, syntetic_target), axis=0)

# Scale the combined training data
X_train_combined_scaled = scaller.fit_transform(X_train_combined)

# Train the model with the best hyperparameters on scaled combined data
best_model = LogisticRegression(**best_params, verbose=0, n_jobs=-1)
best_model.fit(X_train_combined_scaled, y_train_combined)

# Scale the test data
X_test_scaled = scaller.transform(X_test)

# Make predictions
test_predictions = best_model.predict_proba(X_test_scaled)[:, 1]
test_pred_labels = best_model.predict(X_test_scaled)

# Calculate evaluation metrics
test_roc_auc = roc_auc_score(y_test, test_predictions)
test_f1 = f1_score(y_test, test_pred_labels, zero_division=0)
test_precision = precision_score(y_test, test_pred_labels, zero_division=0)
test_recall = recall_score(y_test, test_pred_labels, zero_division=0)
test_accuracy = accuracy_score(y_test, test_pred_labels)

# Create dictionary for test metrics
test_metrics = {
    'test_roc_auc': float(test_roc_auc),
    'test_f1': float(test_f1),
    'test_precision': float(test_precision),
    'test_recall': float(test_recall),
    'test_accuracy': float(test_accuracy)
}

# Print the metrics
print("Test Metrics:")
for metric_name, metric_value in test_metrics.items():
    print(f"{metric_name}: {metric_value:.4f}")

# Save the model and metrics
from joblib import dump
model_filename = 'LogReg_TVAE_test.joblib'
dump(best_model, model_filename)

# Save the test metrics to a file
metrics_and_params = {
    'best_parameters': best_params,
    'test_metrics': test_metrics
}

with open('test_scores_LogReg_TVAE.json', 'w') as f:
    json.dump(metrics_and_params, f, indent=4)



Test Metrics:
test_roc_auc: 0.7791
test_f1: 0.0625
test_precision: 1.0000
test_recall: 0.0323
test_accuracy: 0.9267


# LOGREG + GAUSIAN COPULA

In [69]:
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc
from sklearn.linear_model import LogisticRegression

# Define the best parameters for LogisticRegression (you would normally get these from hyperparameter tuning)
# These are just examples - replace with your actual best parameters
best_params = {
  "C": 0.07416188374104146,
  "max_iter": 200,
  "solver": "sag"
}
# Train the model with the best hyperparameters on X_train_folds
if not isinstance(X_train_folds, pd.DataFrame):
    X_train_df = pd.DataFrame(X_train_folds)
else:
    X_train_df = X_train_folds.copy()

# Add target column to the training data
X_train_df['target'] = y_train_k_fold
X_train_df['index'] = np.arange(1, len(X_train_df) + 1)

# Generate synthetic data using Gaussian Copula
synthesizer = GaussianCopulaSynthesizer(metadata)
synthesizer.fit(X_train_df)
synthetic_data = synthesizer.sample(num_rows=10000)
minority_synthetic_data = synthetic_data[synthetic_data['target'] == 1]
if len(minority_synthetic_data) > 600:
    minority_synthetic_data = minority_synthetic_data.sample(n=600)

# Extract features and target from synthetic data
syntetic_minority_dropped = minority_synthetic_data.copy().drop(['target', 'index'], axis=1)
syntetic_target = minority_synthetic_data['target']

# Combine original and synthetic data
X_train_combined = pd.concat([X_train_df.drop(['target', 'index'], axis=1), syntetic_minority_dropped])
y_train_combined = np.concatenate((y_train_k_fold, syntetic_target), axis=0)

# Scale the combined training data
X_train_combined_scaled = scaller.fit_transform(X_train_combined)

# Train the model with the best hyperparameters on scaled combined data
best_model = LogisticRegression(**best_params, verbose=0, n_jobs=-1)
best_model.fit(X_train_combined_scaled, y_train_combined)

# Scale the test data
X_test_scaled = scaller.transform(X_test)

# Make predictions
test_predictions = best_model.predict_proba(X_test_scaled)[:, 1]
test_pred_labels = best_model.predict(X_test_scaled)

# Calculate evaluation metrics
test_roc_auc = roc_auc_score(y_test, test_predictions)
test_f1 = f1_score(y_test, test_pred_labels, zero_division=0)
test_precision = precision_score(y_test, test_pred_labels, zero_division=0)
test_recall = recall_score(y_test, test_pred_labels, zero_division=0)
test_accuracy = accuracy_score(y_test, test_pred_labels)

# Create dictionary for test metrics
test_metrics = {
    'test_roc_auc': float(test_roc_auc),
    'test_f1': float(test_f1),
    'test_precision': float(test_precision),
    'test_recall': float(test_recall),
    'test_accuracy': float(test_accuracy)
}

# Print the metrics
print("Test Metrics:")
for metric_name, metric_value in test_metrics.items():
    print(f"{metric_name}: {metric_value:.4f}")

# Save the model and metrics
from joblib import dump
model_filename = 'LogReg_GaussianCopula_test.joblib'
dump(best_model, model_filename)

# Save the test metrics to a file
metrics_and_params = {
    'best_parameters': best_params,
    'test_metrics': test_metrics
}

with open('test_scores_LogReg_GaussianCopula.json', 'w') as f:
    json.dump(metrics_and_params, f, indent=4)



Test Metrics:
test_roc_auc: 0.7396
test_f1: 0.1961
test_precision: 0.2500
test_recall: 0.1613
test_accuracy: 0.8998


Test Metrics:
test_roc_auc: 0.7669
test_f1: 0.3125
test_precision: 0.3030
test_recall: 0.3226
test_accuracy: 0.8924

# LOGREG + CV

In [54]:
import importlib
import edge_case
importlib.reload(edge_case)
from edge_case import generate_edge_cases

edge_cases = generate_edge_cases(num_samples=150)
edge_cases

,age,height,clinical_presentation,ef,cerebrovascular_disease,peripheral_artery_disease,if_yes_what_type___1,single_vessel,calcium,medina_side,...,restenosis_reocclusion,adhoc_pci,main_predilatation,stent_diameter,stent_length,mortality,smoking,dyslipidemia,anemia,atrial_fibrilation
0,95.472995,142.906005,4,29.218585,1,1,1,1,1,1,...,1,1,1,2.120859,30.900306,1,1,1,1,1
1,93.600876,158.283900,5,15.121712,1,1,1,0,1,1,...,1,0,1,2.002335,33.075288,1,1,1,1,1
2,96.763787,164.474090,4,23.776688,1,1,1,1,1,1,...,1,0,1,2.531475,35.958950,1,1,1,1,1
3,94.028063,146.571980,5,27.841336,1,1,1,1,1,0,...,0,1,1,2.555159,35.847868,1,1,1,1,1
4,94.185964,155.111592,5,24.433115,1,1,1,1,1,1,...,1,1,1,2.460611,29.386740,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,88.566343,148.171273,5,17.994487,1,0,1,1,1,1,...,1,1,1,2.237761,28.026405,1,1,1,1,1
146,95.455545,160.046251,4,25.877289,1,1,1,1,1,0,...,1,1,0,2.475728,36.354802,1,1,1,1,1
147,86.448409,145.654680,5,22.667131,1,1,1,1,1,1,...,1,1,0,2.113203,29.803520,1,1,1,1,1
148,98.509728,152.623875,4,17.197827,1,1,1,1,1,1,...,1,1,1,2.161004,37.391222,1,1,1,1,0


In [55]:
edge_case_cols = set(edge_cases.columns)
x_train_cols = set(X_train.columns)

common_cols = edge_case_cols.intersection(x_train_cols)
only_in_edge_cases = edge_case_cols - x_train_cols
only_in_x_train = x_train_cols - edge_case_cols

print(f"COMPARISON OF COLUMNS:")
print(f"\nTotal columns in edge_cases: {len(edge_case_cols)}")
print(f"Total columns in X_train: {len(x_train_cols)}")
print(f"Number of common columns: {len(common_cols)}")

print("\n--- COMMON COLUMNS ---")
for col in sorted(common_cols):
    print(f"- {col}")

print("\n--- COLUMNS ONLY IN EDGE CASES ---")
for col in sorted(only_in_edge_cases):
    print(f"- {col}")

print("\n--- COLUMNS ONLY IN X_TRAIN ---")
for col in sorted(only_in_x_train):
    print(f"- {col}")

COMPARISON OF COLUMNS:

Total columns in edge_cases: 30
Total columns in X_train: 22
Number of common columns: 22

--- COMMON COLUMNS ---
- adhoc_pci
- age
- anemia
- atrial_fibrilation
- calcium
- cerebrovascular_disease
- cto_bifurc
- def
- ef
- height
- history_of_cancer
- if_yes_what_type___1
- medina_side
- peripheral_artery_disease
- previous_pci
- previous_stroke_tia
- single_vessel
- stent_diameter
- stent_length
- stent_type___3
- stent_type___4
- stent_type___5

--- COLUMNS ONLY IN EDGE CASES ---
- clinical_presentation
- dyslipidemia
- main_predilatation
- mortality
- restenosis_reocclusion
- side_diametr
- smoking
- trifurcation

--- COLUMNS ONLY IN X_TRAIN ---


In [56]:
edge_cases_target = edge_cases['mortality']
edge_cases_features = edge_cases.copy().drop('mortality', axis=1)

In [57]:
edge_cases_features_names = [feature for feature in edge_cases.columns if feature in X_train.columns]
edge_cases_features = pd.DataFrame(edge_cases, columns=edge_cases_features_names)
edge_cases_features

,age,height,ef,cerebrovascular_disease,peripheral_artery_disease,if_yes_what_type___1,single_vessel,calcium,medina_side,cto_bifurc,...,previous_pci,previous_stroke_tia,stent_type___3,stent_type___4,stent_type___5,adhoc_pci,stent_diameter,stent_length,anemia,atrial_fibrilation
0,95.472995,142.906005,29.218585,1,1,1,1,1,1,1,...,0,1,0,0,1,1,2.120859,30.900306,1,1
1,93.600876,158.283900,15.121712,1,1,1,0,1,1,1,...,1,1,1,0,1,0,2.002335,33.075288,1,1
2,96.763787,164.474090,23.776688,1,1,1,1,1,1,1,...,1,0,0,1,1,0,2.531475,35.958950,1,1
3,94.028063,146.571980,27.841336,1,1,1,1,1,0,1,...,1,1,0,1,1,1,2.555159,35.847868,1,1
4,94.185964,155.111592,24.433115,1,1,1,1,1,1,1,...,1,1,1,1,1,1,2.460611,29.386740,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,88.566343,148.171273,17.994487,1,0,1,1,1,1,1,...,1,1,1,1,1,1,2.237761,28.026405,1,1
146,95.455545,160.046251,25.877289,1,1,1,1,1,0,1,...,1,0,0,1,1,1,2.475728,36.354802,1,1
147,86.448409,145.654680,22.667131,1,1,1,1,1,1,1,...,1,1,1,1,0,1,2.113203,29.803520,1,1
148,98.509728,152.623875,17.197827,1,1,1,1,1,1,0,...,1,0,1,1,1,1,2.161004,37.391222,1,0


In [58]:
X_train_folds = pd.concat([X_train, X_val, edge_cases_features])
y_train_k_fold = np.concatenate((y_train, y_val, edge_cases_target), axis=0)

In [59]:
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
from sklearn.linear_model import LogisticRegression
import numpy as np
import json
from joblib import dump
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

def objective(params):
    # Convert parameters to proper format
    params['C'] = float(params['C'])
    params['max_iter'] = int(params['max_iter'])
    
    classifier = LogisticRegression(
        C=params['C'],
        max_iter=int(params['max_iter']),
        solver=params['solver'],
        verbose=0,
        n_jobs=-1
    )
    
    kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    roc_auc_scores = []
    f1_scores = []
    precision_scores = []
    recall_scores = []
    accuracy_scores = []

    for train_index, test_index in kf.split(X_train_folds, y_train_k_fold):
        not_scaled_X_train_folds = X_train_folds.copy()
        scaled_X_train_folds = scaler.fit_transform(not_scaled_X_train_folds)
        X_test = scaled_X_train_folds[test_index]
        y_test = y_train_k_fold[test_index]

        # Use original training data without synthetic augmentation
        X_train = scaled_X_train_folds[train_index]
        y_train = y_train_k_fold[train_index]
        
        classifier.fit(X_train, y_train)
        predictions = classifier.predict_proba(X_test)[:, 1]
        y_pred = classifier.predict(X_test)
        
        roc_auc = roc_auc_score(y_test, predictions)
        f1 = f1_score(y_test, y_pred, zero_division=0)
        precision = precision_score(y_test, y_pred, zero_division=0)
        recall = recall_score(y_test, y_pred, zero_division=0)
        accuracy = accuracy_score(y_test, y_pred)
        
        roc_auc_scores.append(roc_auc)
        f1_scores.append(f1)
        precision_scores.append(precision)
        recall_scores.append(recall)
        accuracy_scores.append(accuracy)

    mean_metrics = {
        'roc_auc': np.mean(roc_auc_scores),
        'f1': np.mean(f1_scores),
        'precision': np.mean(precision_scores),
        'recall': np.mean(recall_scores),
        'accuracy': np.mean(accuracy_scores),
        'roc_auc_scores': roc_auc_scores,
        'f1_scores': f1_scores,
        'precision_scores': precision_scores,
        'recall_scores': recall_scores,
        'accuracy_scores': accuracy_scores
    }
    
    return {'loss': -mean_metrics['roc_auc'], 'status': STATUS_OK, 'params': params, 'mean_metrics': mean_metrics}

# Define the search space
space = {
    'C': hp.loguniform('C', np.log(0.001), np.log(10)),
    'max_iter': hp.quniform('max_iter', 100, 1000, 100),
    'solver': hp.choice('solver', ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'])
}

trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=100,
            trials=trials)

# Extracting metrics and params from the best trial
best_trial = trials.best_trial['result']
best_params = best_trial['params']
best_metrics = best_trial['mean_metrics']

# Train the model with the best hyperparameters
best_model = LogisticRegression(**best_params, verbose=0)
scaler.fit_transform(X_train_folds)
best_model.fit(X_train_folds, y_train_k_fold)

# Save the best model and metrics
model_filename = 'LogReg_edge_cases_CV.joblib'
dump(best_model, model_filename)

# Evaluate best model on test dataset
# Scale the test data using the same scaler
X_test_scaled = scaler.transform(X_test)

# Make predictions
test_predictions_proba = best_model.predict_proba(X_test_scaled)[:, 1]
test_predictions = best_model.predict(X_test_scaled)

# Calculate metrics on test data
test_metrics = {
    'roc_auc': roc_auc_score(y_test, test_predictions_proba),
    'f1': f1_score(y_test, test_predictions, zero_division=0),
    'precision': precision_score(y_test, test_predictions, zero_division=0),
    'recall': recall_score(y_test, test_predictions, zero_division=0),
    'accuracy': accuracy_score(y_test, test_predictions)
}

print("Test set evaluation:")
for metric, value in test_metrics.items():
    print(f"{metric}: {value:.4f}")

# Save metrics and best parameters to a file
metrics_and_params = {
    'best_parameters': best_params,
    'cross_validation_metrics': best_metrics,
    'test_metrics': test_metrics
}

with open('scores_LogReg_edge_cases_CV.json', 'w') as f:
    json.dump(metrics_and_params, f)

print("Best parameters, model, and evaluation metrics saved together.")

  2%|▏         | 2/100 [00:03<03:29,  2.14s/trial, best loss: -0.8625952989537862]

/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 10.


/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 10.


/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 10.


/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 10.


/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is s

  4%|▍         | 4/100 [00:05<02:06,  1.32s/trial, best loss: -0.8625952989537862]

/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 10.


/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 10.


/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 10.


/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 10.


/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is s

  6%|▌         | 6/100 [00:05<01:10,  1.34trial/s, best loss: -0.8625952989537862]

...

/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 10.


/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 10.


/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 10.


/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 10.


/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is s

 91%|█████████ | 91/100 [00:23<00:01,  5.81trial/s, best loss: -0.8632820784485128]

/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 10.


/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 10.


/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 10.


/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 10.


/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is s

100%|██████████| 100/100 [00:24<00:00,  4.08trial/s, best loss: -0.8632820784485128]
Test set evaluation:
roc_auc: 0.7285
f1: 0.2443
precision: 0.1600
recall: 0.5161
accuracy: 0.7579
Best parameters, model, and evaluation metrics saved together.


/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 10.


/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 10.


/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 10.


/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 10.


/Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning:

'n_jobs' > 1 does not have any effect when 'solver' is s

# LOGREG + edge cases + ARF

In [71]:
import importlib
import edge_case
importlib.reload(edge_case)
from edge_case import generate_edge_cases

edge_cases = generate_edge_cases(num_samples=150)
edge_cases

,age,height,clinical_presentation,ef,cerebrovascular_disease,peripheral_artery_disease,if_yes_what_type___1,single_vessel,calcium,medina_side,...,restenosis_reocclusion,adhoc_pci,main_predilatation,stent_diameter,stent_length,mortality,smoking,dyslipidemia,anemia,atrial_fibrilation
0,90.618102,167.247977,4,26.657204,1,0,1,1,1,1,...,1,1,1,2.341073,29.412235,1,1,1,1,1
1,99.260715,147.186857,5,23.376064,1,1,0,1,0,1,...,1,1,1,2.506090,33.708774,1,1,1,1,1
2,95.979909,144.346846,5,21.363330,1,1,0,1,0,1,...,1,1,1,2.508149,29.852704,1,1,1,1,1
3,93.979877,154.683583,5,28.595316,0,1,1,1,1,1,...,1,1,1,2.279842,30.786431,1,1,1,1,1
4,87.340280,169.569514,5,16.667962,1,0,1,1,1,1,...,1,1,0,2.706269,30.187098,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,85.553304,155.667298,5,22.109425,1,1,1,1,1,1,...,1,0,1,2.483559,30.124473,1,1,1,1,1
146,94.143465,163.099807,5,25.013366,1,1,1,1,1,1,...,1,1,1,2.172370,34.635236,1,1,0,1,1
147,92.540185,146.474631,5,17.584798,1,1,0,1,1,1,...,1,0,1,2.414497,32.111975,1,1,1,1,1
148,85.772181,158.686714,5,17.884335,1,0,1,1,1,1,...,1,1,1,2.279130,32.024063,1,1,1,1,1


In [66]:
edge_case_cols = set(edge_cases.columns)
x_train_cols = set(X_train.columns)

common_cols = edge_case_cols.intersection(x_train_cols)
only_in_edge_cases = edge_case_cols - x_train_cols
only_in_x_train = x_train_cols - edge_case_cols

print(f"COMPARISON OF COLUMNS:")
print(f"\nTotal columns in edge_cases: {len(edge_case_cols)}")
print(f"Total columns in X_train: {len(x_train_cols)}")
print(f"Number of common columns: {len(common_cols)}")

print("\n--- COMMON COLUMNS ---")
for col in sorted(common_cols):
    print(f"- {col}")

print("\n--- COLUMNS ONLY IN EDGE CASES ---")
for col in sorted(only_in_edge_cases):
    print(f"- {col}")

print("\n--- COLUMNS ONLY IN X_TRAIN ---")
for col in sorted(only_in_x_train):
    print(f"- {col}")

COMPARISON OF COLUMNS:

Total columns in edge_cases: 30
Total columns in X_train: 22
Number of common columns: 22

--- COMMON COLUMNS ---
- adhoc_pci
- age
- anemia
- atrial_fibrilation
- calcium
- cerebrovascular_disease
- cto_bifurc
- def
- ef
- height
- history_of_cancer
- if_yes_what_type___1
- medina_side
- peripheral_artery_disease
- previous_pci
- previous_stroke_tia
- single_vessel
- stent_diameter
- stent_length
- stent_type___3
- stent_type___4
- stent_type___5

--- COLUMNS ONLY IN EDGE CASES ---
- clinical_presentation
- dyslipidemia
- main_predilatation
- mortality
- restenosis_reocclusion
- side_diametr
- smoking
- trifurcation

--- COLUMNS ONLY IN X_TRAIN ---


In [67]:
edge_cases_target = edge_cases['mortality']
edge_cases_features = edge_cases.copy().drop('mortality', axis=1)

In [68]:
edge_cases_features_names = [feature for feature in edge_cases.columns if feature in X_train.columns]
edge_cases_features = pd.DataFrame(edge_cases, columns=edge_cases_features_names)
edge_cases_features

,age,height,ef,cerebrovascular_disease,peripheral_artery_disease,if_yes_what_type___1,single_vessel,calcium,medina_side,cto_bifurc,...,previous_pci,previous_stroke_tia,stent_type___3,stent_type___4,stent_type___5,adhoc_pci,stent_diameter,stent_length,anemia,atrial_fibrilation
0,90.618102,167.247977,26.657204,1,0,1,1,1,1,0,...,1,0,1,1,1,1,2.341073,29.412235,1,1
1,99.260715,147.186857,23.376064,1,1,0,1,0,1,1,...,1,1,0,1,0,1,2.506090,33.708774,1,1
2,95.979909,144.346846,21.363330,1,1,0,1,0,1,1,...,0,1,1,0,0,1,2.508149,29.852704,1,1
3,93.979877,154.683583,28.595316,0,1,1,1,1,1,1,...,1,1,1,0,1,1,2.279842,30.786431,1,1
4,87.340280,169.569514,16.667962,1,0,1,1,1,1,0,...,1,1,0,1,0,1,2.706269,30.187098,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,85.553304,155.667298,22.109425,1,1,1,1,1,1,0,...,1,0,1,1,1,0,2.483559,30.124473,1,1
146,94.143465,163.099807,25.013366,1,1,1,1,1,1,1,...,0,0,1,1,0,1,2.172370,34.635236,1,1
147,92.540185,146.474631,17.584798,1,1,0,1,1,1,0,...,1,0,1,0,1,0,2.414497,32.111975,1,1
148,85.772181,158.686714,17.884335,1,0,1,1,1,1,0,...,1,0,1,1,0,1,2.279130,32.024063,1,1


In [69]:
X_train_folds = pd.concat([X_train, X_val, edge_cases_features])
y_train_k_fold = np.concatenate((y_train, y_val, edge_cases_target), axis=0)

In [72]:
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
from sklearn.linear_model import LogisticRegression
import numpy as np
import pandas as pd
import json
from joblib import dump
import synthcity.logger as log
from synthcity.plugins import Plugins
from synthcity.plugins.core.dataloader import GenericDataLoader

scaler = StandardScaler()

def objective(params):
    # Convert parameters to proper format
    params['C'] = float(params['C'])
    params['max_iter'] = int(params['max_iter'])
    
    classifier = LogisticRegression(
        C=params['C'],
        max_iter=int(params['max_iter']),
        solver=params['solver'],
        verbose=0,
        n_jobs=-1
    )
    
    kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    roc_auc_scores = []
    f1_scores = []
    precision_scores = []
    recall_scores = []
    accuracy_scores = []

    for train_index, test_index in kf.split(X_train_folds, y_train_k_fold):
        not_scaled_X_train_folds = X_train_folds.copy()
        scaled_X_train_folds = scaler.fit_transform(not_scaled_X_train_folds)
        X_test = scaled_X_train_folds[test_index]
        y_test = y_train_k_fold[test_index]

        # Prepare data for ARF
        train = not_scaled_X_train_folds.iloc[train_index].copy().reset_index(drop=True)
        train['target'] = y_train_k_fold[train_index]
        
        # Create a data loader for synthcity
        loader = GenericDataLoader(
            train,
            target_column="target",
        )
        
        # Initialize and fit the ARF model
        syn_model = Plugins().get("arf")
        syn_model.fit(loader)
        
        # Generate synthetic data
        synthetic_data = syn_model.generate(count=10000).dataframe()
        
        # Extract minority class samples
        minority_synthetic_data = synthetic_data[synthetic_data['target'] == 1]
        if len(minority_synthetic_data) > 600:
            minority_synthetic_data = minority_synthetic_data.sample(n=600)
        
        # Extract features from synthetic data
        syntetic_minority_dropped = minority_synthetic_data.drop('target', axis=1)
        syntetic_target = minority_synthetic_data['target'].values

        # Combine original and synthetic data
        X_train = pd.concat([not_scaled_X_train_folds.iloc[train_index], syntetic_minority_dropped])
        X_train = scaler.transform(X_train)
        y_train = np.concatenate((y_train_k_fold[train_index], syntetic_target), axis=0)
        
        classifier.fit(X_train, y_train)
        predictions = classifier.predict_proba(X_test)[:, 1]
        y_pred = classifier.predict(X_test)
        
        roc_auc = roc_auc_score(y_test, predictions)
        f1 = f1_score(y_test, y_pred, zero_division=0)
        precision = precision_score(y_test, y_pred, zero_division=0)
        recall = recall_score(y_test, y_pred, zero_division=0)
        accuracy = accuracy_score(y_test, y_pred)
        
        roc_auc_scores.append(roc_auc)
        f1_scores.append(f1)
        precision_scores.append(precision)
        recall_scores.append(recall)
        accuracy_scores.append(accuracy)

    mean_metrics = {
        'roc_auc': np.mean(roc_auc_scores),
        'f1': np.mean(f1_scores),
        'precision': np.mean(precision_scores),
        'recall': np.mean(recall_scores),
        'accuracy': np.mean(accuracy_scores),
        'roc_auc_scores': roc_auc_scores,
        'f1_scores': f1_scores,
        'precision_scores': precision_scores,
        'recall_scores': recall_scores,
        'accuracy_scores': accuracy_scores
    }
    
    return {'loss': -mean_metrics['roc_auc'], 'status': STATUS_OK, 'params': params, 'mean_metrics': mean_metrics}

# Define the search space
space = {
    'C': hp.loguniform('C', np.log(0.001), np.log(10)),
    'max_iter': hp.quniform('max_iter', 100, 1000, 100),
    'solver': hp.choice('solver', ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'])
}

trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10,
            trials=trials)

# Extracting metrics and params from the best trial
best_trial = trials.best_trial['result']
best_params = best_trial['params']
best_metrics = best_trial['mean_metrics']

# Train the final model with ARF augmentation
# Prepare the data for ARF
train_df = X_train_folds.copy() if isinstance(X_train_folds, pd.DataFrame) else pd.DataFrame(X_train_folds)
train_df['target'] = y_train_k_fold

# Create a data loader for synthcity
loader = GenericDataLoader(
    train_df,
    target_column="target",
)

# Initialize and fit the ARF model
syn_model = Plugins().get("arf")
syn_model.fit(loader)

# Generate synthetic data
synthetic_data = syn_model.generate(count=10000).dataframe()

# Extract minority class samples
minority_synthetic_data = synthetic_data[synthetic_data['target'] == 1]
if len(minority_synthetic_data) > 600:
    minority_synthetic_data = minority_synthetic_data.sample(n=600)

# Extract features and target from synthetic data
syntetic_minority_features = minority_synthetic_data.drop('target', axis=1)
syntetic_target = minority_synthetic_data['target'].values

# Combine original and synthetic data
X_train_combined = pd.concat([train_df.drop('target', axis=1), syntetic_minority_features])
y_train_combined = np.concatenate((y_train_k_fold, syntetic_target), axis=0)

# Scale the combined training data
X_train_combined_scaled = scaler.fit_transform(X_train_combined)

# Train the model with the best hyperparameters
best_model = LogisticRegression(
    C=float(best_params['C']),
    max_iter=int(best_params['max_iter']),
    solver=best_params['solver'],
    verbose=0,
    n_jobs=-1
)
best_model.fit(X_train_combined_scaled, y_train_combined)

# Save the best model and metrics
model_filename = 'LogReg_ARF_CV.joblib'
dump(best_model, model_filename)

# Save metrics and best parameters to a file
metrics_and_params = {
    'best_parameters': best_params,
    'evaluation_metrics': best_metrics
}

with open('scores_LogReg_ARF_edge_cases.json', 'w') as f:
    json.dump(metrics_and_params, f, indent=4)

print("Best parameters, model, and evaluation metrics saved together.")

# Evaluate the best model on test dataset
print("Evaluating best model on test dataset...")

# Make sure to apply the same preprocessing to test data as was done for training data
X_test_scaled = scaler.transform(X_test)  # Assuming X_test is your test dataset

# Make predictions
test_predictions = best_model.predict_proba(X_test_scaled)[:, 1]
test_pred_labels = best_model.predict(X_test_scaled)

# Calculate evaluation metrics
test_roc_auc = roc_auc_score(y_test, test_predictions)
test_f1 = f1_score(y_test, test_pred_labels, zero_division=0)
test_precision = precision_score(y_test, test_pred_labels, zero_division=0)
test_recall = recall_score(y_test, test_pred_labels, zero_division=0)
test_accuracy = accuracy_score(y_test, test_pred_labels)

# Create dictionary for test metrics
test_metrics = {
    'test_roc_auc': float(test_roc_auc),
    'test_f1': float(test_f1),
    'test_precision': float(test_precision),
    'test_recall': float(test_recall),
    'test_accuracy': float(test_accuracy)
}

print("Test Metrics:")
for metric_name, metric_value in test_metrics.items():
    print(f"{metric_name}: {metric_value:.4f}")

# Add test metrics to the saved JSON file
metrics_and_params['test_metrics'] = test_metrics

# Save updated metrics including test results
with open('scores_LogReg_ARF_edge_cases.json', 'w') as f:
    json.dump(metrics_and_params, f, indent=4)

print("Model evaluation on test dataset completed and results saved.")

[KeOps] Warning : omp.h header is not in the path, disabling OpenMP. To fix this, you can set the environment
                  variable OMP_PATH to the location of the header before importing keopscore or pykeops,
                  e.g. using os.environ: import os; os.environ['OMP_PATH'] = '/path/to/omp/header'
[KeOps] Warning : Cuda libraries were not detected on the system or could not be loaded ; using cpu only mode
  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

[2025-05-27T18:56:31.102307+0400][30336][CRITICAL] load failed: Failed to import transformers.trainer because of the following error (look up to see its traceback):
Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
Failed to import transformers.modeling_tf_utils because of the following error (look up to see its traceback):
Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.
[2025-05-27T18:56:31.102901+0400][30336][CRITICAL] load failed: module 'synthcity.plugins.generic.plugin_great' has no attribute 'plugin'
[2025-05-27T18:56:31.103102+0400][30336][CRITICAL] module plugin_great load failed
[2025-05-27T18:56:31.216868+0400][30336][CRITICAL] module disabled: /Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/synthcity/plugins/generic/plugin_goggle.py


Initial accuracy is 0.7475093399750934                
Iteration number 1 reached accuracy of 0.44956413449564137.
  0%|          | 0/10 [00:05<?, ?trial/s, best loss=?]

[2025-05-27T18:56:50.734937+0400][30336][CRITICAL] load failed: module 'synthcity.plugins.generic.plugin_great' has no attribute 'plugin'

[2025-05-27T18:56:50.736380+0400][30336][CRITICAL] load failed: module 'synthcity.plugins.generic.plugin_great' has no attribute 'plugin'

[2025-05-27T18:56:50.736680+0400][30336][CRITICAL] module plugin_great load failed

[2025-05-27T18:56:50.736991+0400][30336][CRITICAL] module disabled: /Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/synthcity/plugins/generic/plugin_goggle.py



Initial accuracy is 0.7649439601494396                
Iteration number 1 reached accuracy of 0.43555417185554174.
  0%|          | 0/10 [00:23<?, ?trial/s, best loss=?]

...

Initial accuracy is 0.7457996266334785                                            
Iteration number 1 reached accuracy of 0.4290603609209708.                        
 90%|█████████ | 9/10 [26:46<02:42, 162.88s/trial, best loss: -0.8663028754741896]

[2025-05-27T19:23:30.164831+0400][30336][CRITICAL] load failed: module 'synthcity.plugins.generic.plugin_great' has no attribute 'plugin'

[2025-05-27T19:23:30.165315+0400][30336][CRITICAL] load failed: module 'synthcity.plugins.generic.plugin_great' has no attribute 'plugin'

[2025-05-27T19:23:30.165555+0400][30336][CRITICAL] module plugin_great load failed

[2025-05-27T19:23:30.165848+0400][30336][CRITICAL] module disabled: /Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/synthcity/plugins/generic/plugin_goggle.py



Initial accuracy is 0.7585563161169881                                            
Iteration number 1 reached accuracy of 0.4315494710640946.                        
100%|██████████| 10/10 [27:16<00:00, 163.62s/trial, best loss: -0.8663028754741896]

[2025-05-27T19:23:46.929294+0400][30336][CRITICAL] load failed: module 'synthcity.plugins.generic.plugin_great' has no attribute 'plugin'

[2025-05-27T19:23:46.929638+0400][30336][CRITICAL] load failed: module 'synthcity.plugins.generic.plugin_great' has no attribute 'plugin'

[2025-05-27T19:23:46.930268+0400][30336][CRITICAL] module plugin_great load failed

[2025-05-27T19:23:46.930692+0400][30336][CRITICAL] module disabled: /Users/ivan.petrov/HSE/.venv/lib/python3.12/site-packages/synthcity/plugins/generic/plugin_goggle.py




Initial accuracy is 0.7565826330532213
Iteration number 1 reached accuracy of 0.44425770308123247.
Best parameters, model, and evaluation metrics saved together.
Evaluating best model on test dataset...
Test Metrics:
test_roc_auc: 0.7739
test_f1: 0.3019
test_precision: 0.3636
test_recall: 0.2581
test_accuracy: 0.9095
Model evaluation on test dataset completed and results saved.
